Make necessary installs

In [1]:
!pip install transformers peft datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.model_selection import train_test_split
import pandas as pd


In [6]:
# Load 20 Newsgroups dataset and filter specific categories
dataset = load_dataset("TopicNet/20-Newsgroups")

# Select distinct categories
selected_categories = ['sci.space', 'rec.autos', 'comp.graphics', 'talk.politics.mideast']
dataset = dataset.filter(lambda example: example['topic'] in selected_categories)

# Map categories to numerical labels
label_mapping = {label: i for i, label in enumerate(selected_categories)}
dataset = dataset.map(lambda example: {'label': label_mapping[example['topic']]})
dataset = dataset.rename_column('text', 'sentence')

# Split into train and test sets
dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset['train']
eval_dataset = dataset['test']

# Show a sample
print(train_dataset[0])

{'Unnamed: 0': 1,
 'raw_text': "A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.",
 'filenames': '/home/egorov/scikit_learn_data/20news_home/20news-bydate-train/comp.sys.mac.hardware/51861',
 'target': 4,
 'id': 'comp_sys_mac_hardware_51861',
 'tokenized': "[('fair', 'JJ'), ('number', 'NN'), ('of', 'IN'), ('brave', 'JJ'), ('souls', 'NNS'), ('who', 'WP'), ('upgraded', 'VBD'), ('their', 'PRP$'), ('si', 'NN'), ('clock', 'NN'), ('oscillator', 'NN'), ('have', 'VBP'), ('shared', 'VBN'), ('their', 'PRP$'), ('exp

In [ ]:
# Load tokenizer for LLaMA model
MODEL_NAME = "meta-llama/Meta-Llama-3-1B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token if not available

def tokenize_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

In [ ]:
# Load base model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(selected_categories),
    device_map="auto"
)

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"]  # Apply LoRA to query and value layers
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True
)

In [ ]:
# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=None  # Add a custom metric function later if desired
)


In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

In [ ]:
# Save the model and tokenizer
model.save_pretrained("./fine-tuned-llama3-1B-topic-classification")
tokenizer.save_pretrained("./fine-tuned-llama3-1B-topic-classification")